# 客户购买预测数据分析

## 数据概述
本数据集包含客户的基本信息和购买行为，用于构建购买预测模型。

### 数据字段
- **Country**: 国家
- **Age**: 年龄
- **Salary**: 薪资
- **Purchased**: 是否购买 (Yes/No)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc

# 设置显示中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 读取数据
df = pd.read_csv('Customer Purchase Prediction Dataset.csv')

# 显示前几行数据
print("数据集形状:", df.shape)
print("\n前5行数据:")
df.head()

In [ ]:
# 数据基本信息
print("数据类型:")
print(df.dtypes)
print("\n缺失值统计:")
print(df.isnull().sum())
print("\n数据描述统计:")
print(df.describe())

## 数据分析结果

### 1. 数据预处理

In [ ]:
# 处理缺失值
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Salary'].fillna(df['Salary'].median(), inplace=True)

# 编码分类变量
le_country = LabelEncoder()
df['Country_encoded'] = le_country.fit_transform(df['Country'])

le_purchased = LabelEncoder()
df['Purchased_encoded'] = le_purchased.fit_transform(df['Purchased'])

print("编码映射:")
print("国家编码:", dict(zip(le_country.classes_, le_country.transform(le_country.classes_))))
print("购买编码:", dict(zip(le_purchased.classes_, le_purchased.transform(le_purchased.classes_))))

# 显示处理后的数据
print("\n处理后的数据:")
df.head()

### 2. 购买行为分析

In [ ]:
# 购买率统计
purchase_counts = df['Purchased'].value_counts()
purchase_rate = purchase_counts['Yes'] / len(df) * 100

print("购买行为统计:")
print(purchase_counts)
print(f"\n购买率: {purchase_rate:.2f}%")

In [ ]:
# 购买率可视化
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 饼图
colors = ['#66b3ff', '#ff9999']
axes[0].pie(purchase_counts.values, labels=purchase_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)
axes[0].set_title('购买行为分布')

# 柱状图
purchase_counts.plot(kind='bar', ax=axes[1], color=colors)
axes[1].set_title('购买行为数量')
axes[1].set_ylabel('数量')
axes[1].set_xlabel('是否购买')
axes[1].tick_params(axis='x', rotation=0)
axes[1].set_xticklabels(['不购买', '购买'])

plt.tight_layout()
plt.show()

### 3. 人口统计特征分析

In [ ]:
# 年龄统计
age_stats = df.groupby('Purchased')['Age'].agg(['mean', 'median', 'std'])
print("按购买行为分组的年龄统计:")
print(age_stats)

In [ ]:
# 薪资统计
salary_stats = df.groupby('Purchased')['Salary'].agg(['mean', 'median', 'std'])
print("按购买行为分组的薪资统计:")
print(salary_stats)

In [ ]:
# 年龄和薪资分布对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 年龄分布
for purchased in ['Yes', 'No']:
    sns.kdeplot(df[df['Purchased'] == purchased]['Age'], label=purchased, ax=axes[0])
axes[0].set_title('年龄分布对比')
axes[0].set_xlabel('年龄')
axes[0].set_ylabel('密度')
axes[0].legend()

# 薪资分布
for purchased in ['Yes', 'No']:
    sns.kdeplot(df[df['Purchased'] == purchased]['Salary'], label=purchased, ax=axes[1])
axes[1].set_title('薪资分布对比')
axes[1].set_xlabel('薪资')
axes[1].set_ylabel('密度')
axes[1].legend()

plt.tight_layout()
plt.show()

### 4. 地理位置分析

In [ ]:
# 按国家统计
country_stats = df.groupby('Country')['Purchased'].agg(['count', lambda x: (x == 'Yes').mean()])
country_stats.columns = ['总客户数', '购买率']
country_stats['购买率'] = (country_stats['购买率'] * 100).round(2)

print("各国购买行为统计:")
print(country_stats)

In [ ]:
# 国家购买率可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 总客户数
country_stats['总客户数'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('各国客户数量')
axes[0].set_ylabel('客户数')
axes[0].tick_params(axis='x', rotation=0)

# 购买率
country_stats['购买率'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('各国购买率')
axes[1].set_ylabel('购买率 (%)')
axes[1].tick_params(axis='x', rotation=0)
axes[1].axhline(y=purchase_rate, color='red', linestyle='--', label=f'平均: {purchase_rate:.1f}%')
axes[1].legend()

plt.tight_layout()
plt.show()

### 5. 特征关系分析

In [ ]:
# 年龄与薪资关系（按购买行为分组）
plt.figure(figsize=(10, 6))
colors = {'Yes': 'green', 'No': 'red'}
for purchased in ['Yes', 'No']:
    subset = df[df['Purchased'] == purchased]
    plt.scatter(subset['Age'], subset['Salary'], 
               c=colors[purchased], label=purchased, alpha=0.6)
plt.xlabel('年龄')
plt.ylabel('薪资')
plt.title('年龄与薪资关系（按购买行为着色）')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 6. 特征重要性分析

In [ ]:
# 使用随机森林评估特征重要性
X = df[['Country_encoded', 'Age', 'Salary']]
y = df['Purchased_encoded']

# 训练随机森林模型
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# 特征重要性
feature_importance = pd.DataFrame({
    '特征': ['Country', 'Age', 'Salary'],
    '重要性': rf.feature_importances_
}).sort_values('重要性', ascending=False)

print("特征重要性:")
print(feature_importance)

In [ ]:
# 特征重要性可视化
plt.figure(figsize=(8, 5))
plt.bar(feature_importance['特征'], feature_importance['重要性'], color='steelblue')
plt.title('特征重要性排序')
plt.ylabel('重要性')
plt.xlabel('特征')
plt.tight_layout()
plt.show()

### 7. 预测模型构建

In [ ]:
# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 训练逻辑回归模型
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# 训练决策树模型
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# 训练随机森林模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 预测
y_pred_lr = lr.predict(X_test)
y_pred_dt = dt.predict(X_test)
y_pred_rf = rf_model.predict(X_test)

# 模型性能对比
models = {
    '逻辑回归': y_pred_lr,
    '决策树': y_pred_dt,
    '随机森林': y_pred_rf
}

print("模型性能对比:")
for model_name, y_pred in models.items():
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name}: 准确率 = {accuracy:.4f}")

In [ ]:
# 混淆矩阵可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (model_name, y_pred) in enumerate(models.items()):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'{model_name} 混淆矩阵')
    axes[i].set_ylabel('实际')
    axes[i].set_xlabel('预测')

plt.tight_layout()
plt.show()

In [ ]:
# ROC曲线
plt.figure(figsize=(10, 6))

for model_name, model in [('逻辑回归', lr), ('决策树', dt), ('随机森林', rf_model)]:
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('假阳性率')
plt.ylabel('真阳性率')
plt.title('ROC曲线对比')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 数据场景

### 1. 精准营销
- **应用场景**: 根据客户特征预测购买可能性，进行精准营销
- **价值**: 提高营销ROI，降低获客成本

### 2. 客户分层
- **应用场景**: 根据购买概率和特征对客户进行分层
- **价值**: 为不同层级客户提供差异化服务和产品

### 3. 产品推荐
- **应用场景**: 基于客户特征推荐合适的产品
- **价值**: 提高转化率，增强用户粘性

### 4. 风险管理
- **应用场景**: 识别低购买概率客户，优化资源配置
- **价值**: 减少无效投入，提高运营效率

### 5. 市场拓展
- **应用场景**: 分析不同国家的购买行为，指导市场拓展策略
- **价值**: 选择高潜力市场，优化国际化布局

### 6. 定价策略
- **应用场景**: 分析薪资与购买行为关系，优化定价策略
- **价值**: 找到最优定价点，平衡销量和利润

## 主要发现

1. **购买率**: 整体购买率反映市场需求水平
2. **年龄影响**: 年龄与购买行为存在明显相关性
3. **薪资影响**: 薪资水平对购买决策有重要影响
4. **地区差异**: 不同国家的购买率存在差异
5. **特征重要性**: 年龄和薪资是预测购买行为的最重要因素
6. **模型性能**: 随机森林模型表现最佳

## 数据质量
- 数据完整性: 部分记录存在缺失值，已进行填充
- 数据准确性: 数据基本准确，可用于建模
- 数据时效性: 需要定期更新以反映最新市场状况

## 建议

1. **营销策略**: 重点针对高购买概率客户进行营销
2. **客户分层**: 建立客户分层体系，提供差异化服务
3. **产品定价**: 根据薪资水平制定差异化定价策略
4. **市场拓展**: 优先进入购买率较高的国家
5. **模型优化**: 持续收集数据，优化预测模型
6. **特征工程**: 考虑引入更多特征提高预测精度